In [1]:
import pandas as pd
import numpy as np
import os
import pickle

## Load results 

In [2]:
with open('results_summary.dict', 'rb') as f:
    stats = pickle.load(f)

## Main tables

In [3]:
def get_latex(df):
    print(df.to_latex().replace('\$', '$'))

In [4]:
def get_results_table(results_dict, metrics, keep_methods):
    d = {}
    missing_methods = []
    for method in keep_methods:
        if method in results_dict:
            d[method] = ['{:.3f} ({:.3f})'.format(*results_dict[method][metric]) if 'time' in metric else \
                         '{:.2f} ({:.2f})'.format(*results_dict[method][metric]) for metric in metrics]
        else:
            missing_methods.append(method)
    df = pd.DataFrame.from_dict(d, orient='index')
    df.columns = metrics
    df.index.names = ['alg']
    if len(missing_methods) > 0:
        print('missing methods: {}'.format(missing_methods))
    return df.loc[[x for x in keep_methods if x not in missing_methods]], missing_methods

In [5]:
def get_full_results_table(results_dict, exper_type, num=5):
    metrics = ['test_eval', 
               'test_eq_max', 'test_eq_mean', 
               'test_ineq_max', 'test_ineq_mean',
               'test_time']
    metrics_raw = [x.replace('test_', 'test_raw_') for x in metrics]
    metrics_renaming = ['Obj. value', 
                        'Max eq viol.', 'Mean eq viol.', 
                        'Max ineq viol.', 'Mean ineq viol.',
                        'Time']
    
    nn_methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 
               'baseline_nn', 'baseline_eq_nn']
    opt_methods = dict([
            ('simple', ['osqp', 'qpth']), ('nonconvex', ['ipopt']), ('acopf', ['pypower'])
    ])
    opt_methods_list = ['baseline_opt_{}'.format(x) for x in opt_methods[exper_type]]
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            ('baseline_opt_osqp', 'Optimizer (OSQP)'),
            ('baseline_opt_qpth', 'Optimizer (qpth)'),
            ('baseline_opt_ipopt', 'Optimizer (IPOPT)'),
            ('baseline_opt_pypower', 'Optimizer (PYPOWER)')
        ))
    
    df1, missing_methods = get_results_table(results_dict, metrics, nn_methods + opt_methods_list)
    
    df2, missing_methods_2 = get_results_table(results_dict, metrics_raw, nn_methods)
    df2.columns = metrics
    df2.index =  ['{}-noTestCorr'.format(x) for x in df2.index]
    
    missing_methods = missing_methods + ['{}-noTestCorr'.format(x) for x in missing_methods_2]
        
    all_results = pd.concat([df1, df2])
    results_ordering = opt_methods_list + ['method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
                       'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
                       'baseline_eq_nn']
    all_results = all_results.loc[[x for x in results_ordering if x not in missing_methods]]
    all_results.index = [methods_renaming_dict[x] for x in all_results.index]
    all_results.columns = metrics_renaming
    
    return all_results

In [6]:
df_simple_5050 = get_full_results_table(stats['simple_ineq50_eq50'], exper_type='simple')
df_simple_5050

,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
Optimizer (OSQP),-15.05 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.002 (0.000)
Optimizer (qpth),-15.05 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),1.335 (0.012)
DC3,-13.46 (0.01),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.017 (0.001)
"DC3, $\neq$",-12.58 (0.04),0.35 (0.00),0.13 (0.00),0.00 (0.00),0.00 (0.00),0.008 (0.000)
"DC3, $\not\leq$ train",-1.39 (0.97),0.00 (0.00),0.00 (0.00),0.02 (0.02),0.00 (0.00),0.017 (0.000)
"DC3, $\not\leq$ train/test",-1.23 (1.21),0.00 (0.00),0.00 (0.00),0.09 (0.13),0.01 (0.01),0.001 (0.000)
"DC3, no soft loss",-21.84 (0.00),0.00 (0.00),0.00 (0.00),23.83 (0.11),4.04 (0.01),0.017 (0.000)
NN,-12.57 (0.01),0.35 (0.00),0.13 (0.00),0.00 (0.00),0.00 (0.00),0.001 (0.000)
"NN, $\leq$ test",-12.57 (0.01),0.35 (0.00),0.13 (0.00),0.00 (0.00),0.00 (0.00),0.008 (0.000)
Eq.~NN,-9.16 (0.75),0.00 (0.00),0.00 (0.00),8.83 (0.72),0.91 (0.09),0.001 (0.000)


In [7]:
df_nonconvex = get_full_results_table(stats['nonconvex'], exper_type='nonconvex')
df_nonconvex

,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
Optimizer (IPOPT),-11.59 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.121 (0.000)
DC3,-10.66 (0.03),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.013 (0.000)
"DC3, $\neq$",-10.04 (0.02),0.35 (0.00),0.13 (0.00),0.00 (0.00),0.00 (0.00),0.009 (0.000)
"DC3, $\not\leq$ train",-0.29 (0.67),0.00 (0.00),0.00 (0.00),0.01 (0.01),0.00 (0.00),0.010 (0.004)
"DC3, $\not\leq$ train/test",-0.27 (0.67),0.00 (0.00),0.00 (0.00),0.03 (0.03),0.00 (0.00),0.001 (0.000)
"DC3, no soft loss",-13.81 (0.00),0.00 (0.00),0.00 (0.00),15.21 (0.04),2.33 (0.01),0.013 (0.000)
NN,-10.02 (0.01),0.35 (0.00),0.13 (0.00),0.00 (0.00),0.00 (0.00),0.001 (0.000)
"NN, $\leq$ test",-10.02 (0.01),0.35 (0.00),0.13 (0.00),0.00 (0.00),0.00 (0.00),0.009 (0.000)
Eq.~NN,-3.88 (0.56),0.00 (0.00),0.00 (0.00),6.87 (0.43),0.72 (0.05),0.001 (0.000)
"Eq.~NN, $\leq$ test",-10.99 (0.03),0.00 (0.00),0.00 (0.00),0.87 (0.04),0.06 (0.00),0.013 (0.000)


In [8]:
df_acopf = get_full_results_table(stats['acopf'], exper_type='acopf')
df_acopf

,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
Optimizer (PYPOWER),3.81 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.949 (0.002)
DC3,3.82 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.089 (0.000)
"DC3, $\neq$",3.67 (0.01),0.14 (0.01),0.02 (0.00),0.00 (0.00),0.00 (0.00),0.040 (0.000)
"DC3, $\not\leq$ train",3.82 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.089 (0.000)
"DC3, $\not\leq$ train/test",3.82 (0.00),0.00 (0.00),0.00 (0.00),0.01 (0.00),0.00 (0.00),0.039 (0.000)
"DC3, no soft loss",3.11 (0.05),2.60 (0.35),0.07 (0.00),2.33 (0.33),0.03 (0.01),0.088 (0.000)
NN,3.69 (0.02),0.19 (0.01),0.03 (0.00),0.00 (0.00),0.00 (0.00),0.001 (0.000)
"NN, $\leq$ test",3.69 (0.02),0.16 (0.00),0.02 (0.00),0.00 (0.00),0.00 (0.00),0.040 (0.000)
Eq.~NN,3.81 (0.00),0.00 (0.00),0.00 (0.00),0.15 (0.01),0.00 (0.00),0.039 (0.000)
"Eq.~NN, $\leq$ test",3.81 (0.00),0.00 (0.00),0.00 (0.00),0.15 (0.01),0.00 (0.00),0.078 (0.000)


## Appendix tables (simple problem)

In [9]:
def get_comparison_table(all_stats, experiments, metrics, keep_methods):
    d = {}
    for method in keep_methods:
        for metric in metrics:
            d[(method, metric)] = {}
            for experiment in experiments:
                d[(method, metric)][experiment] = '{:.2f} ({:.2f})'.format(*all_stats[experiment][method][metric])
        df = pd.DataFrame.from_dict(d, orient='index')           
    df.index.names = ['Alg', 'Metric']
    return df.iloc[df.index.isin(keep_methods, 'Alg')]

In [10]:
def get_simple_comparison_table(all_stats, vary_across):
    assert vary_across in ['ineq', 'eq'], 'vary_across should be in [ineq, eq]'
    
    if vary_across == 'ineq':
        experiments = ['simple_ineq10_eq50', 'simple_ineq30_eq50', 'simple_ineq50_eq50', 'simple_ineq70_eq50',
                       'simple_ineq90_eq50']
        experiments_renaming = ['10', '30', '50', '70', '90']
    else:
        experiments = ['simple_ineq50_eq10', 'simple_ineq50_eq30', 'simple_ineq50_eq50', 'simple_ineq50_eq70',
                       'simple_ineq50_eq90']
        experiments_renaming = ['10', '30', '50', '70', '90']
    
    methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 
               'baseline_nn', 'baseline_eq_nn', 'baseline_opt_osqp']
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            ('baseline_opt_osqp', 'Optimizers (OSQP, qpth)')
        ))
    
    metrics = ['test_eval', 'test_eq_max', 'test_ineq_max']
    metrics_renaming_dict = dict((('test_eval', 'Obj. val.'), ('test_eq_max', 'Max eq.'), ('test_ineq_max', 'Max ineq.')  ))
    clean_metrics = [metrics_renaming_dict[x] for x in metrics]
    metrics_raw = [x.replace('test_', 'test_raw_') if 'time' not in x else x for x in metrics]
    
    df1 = get_comparison_table(all_stats, experiments, metrics, methods)
    df1 = df1.reset_index()
    df1['Metric'] = df1['Metric'].str.replace('_obj_val', '_eval') # make naming consistent
    df1 = df1.set_index(['Alg', 'Metric'])
    
    df2 = get_comparison_table(all_stats, experiments, metrics_raw, methods[:-1])
    df2.columns = experiments
    df2 = df2.reset_index()
    df2['Alg'] = ['{}-noTestCorr'.format(x) for x in df2['Alg']]
    df2['Metric'] = df2['Metric'].str.replace('raw_', '')  # make naming consistent
    df2 = df2.set_index(['Alg', 'Metric'])
    
    all_results = pd.concat([df1, df2])
    results_ordering = ['baseline_opt_osqp', 'method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
                       'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
                       'baseline_eq_nn']
    clean_results_ordering = [methods_renaming_dict[x] for x in results_ordering]
    all_results = all_results.iloc[all_results.index.isin(results_ordering, 'Alg')]
    all_results = all_results.reindex(pd.MultiIndex.from_product([results_ordering, metrics], names=['Alg', 'Metric']))
    all_results = all_results.reset_index()
    all_results['Alg'] = [methods_renaming_dict[x] for x in all_results['Alg']]
    all_results['Metric'] = [metrics_renaming_dict[x] for x in all_results['Metric']]
    all_results = all_results.set_index(['Alg', 'Metric'])
    all_results.columns = experiments_renaming
    all_results = all_results.reindex(pd.MultiIndex.from_product([clean_results_ordering, clean_metrics])) # remove names
    
    return all_results

In [11]:
df_simple_varyeq = get_simple_comparison_table(stats, 'eq')
df_simple_varyeq

10             30  \
Optimizers (OSQP, qpth)    Obj. val.  -27.26 (0.00)  -23.13 (0.00)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3                        Obj. val.  -25.79 (0.02)  -20.29 (0.20)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\neq$                Obj. val.  -22.59 (0.18)  -20.40 (0.03)   
                           Max eq.      0.12 (0.01)    0.24 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\not\leq$ train      Obj. val.  -25.75 (0.04)  -20.14 (0.15)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\not\leq$ train/test Obj. val.  -25.75 (0.04)  -20.14 (0.15)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, no soft loss          Obj. val.  -66.79 (0.01)  -40.63 (0.02)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.  122.52 (0.22)   50.25 (0.14)   
NN                         Obj. val.  -22.65 (0.12)  -20.43 (0.03)   
                           Max eq.      0.11 (0.01)    0.24 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
NN, $\leq$ test            Obj. val.  -22.65 (0.12)  -20.43 (0.03)   
                           Max eq.      0.11 (0.01)    0.24 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
Eq.~NN                     Obj. val.  -27.20 (0.02)  -22.74 (0.14)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.43 (0.02)    1.37 (0.09)   
Eq.~NN, $\leq$ test        Obj. val.  -27.20 (0.02)  -22.76 (0.13)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.42 (0.02)    1.27 (0.09)   

                                                 50             70  \
Optimizers (OSQP, qpth)    Obj. val.  -15.05 (0.00)  -14.80 (0.00)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3                        Obj. val.  -13.46 (0.01)  -13.73 (0.02)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\neq$                Obj. val.  -12.58 (0.04)  -13.36 (0.02)   
                           Max eq.      0.35 (0.00)    0.47 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\not\leq$ train      Obj. val.   -1.39 (0.97)  -13.71 (0.04)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.02 (0.02)    0.00 (0.00)   
DC3, $\not\leq$ train/test Obj. val.   -1.23 (1.21)  -13.71 (0.04)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.09 (0.13)    0.00 (0.00)   
DC3, no soft loss          Obj. val.  -21.84 (0.00)  -15.56 (0.02)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.   23.83 (0.11)    5.85 (0.18)   
NN                         Obj. val.  -12.57 (0.01)  -13.35 (0.03)   
                           Max eq.      0.35 (0.00)    0.47 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
NN, $\leq$ test            Obj. val.  -12.57 (0.01)  -13.35 (0.03)   
                           Max eq.      0.35 (0.00)    0.47 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
Eq.~NN                     Obj. val.   -9.16 (0.75)  -14.64 (0.01)   
                           Max eq.      0.00 (0.00)    0.0

In [12]:
df_simple_varyineq = get_simple_comparison_table(stats, 'ineq')
df_simple_varyineq

10             30  \
Optimizers (OSQP, qpth)    Obj. val.  -17.33 (0.00)  -16.33 (0.00)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3                        Obj. val.  -15.18 (0.31)  -13.90 (0.20)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\neq$                Obj. val.  -15.66 (0.05)  -14.10 (0.04)   
                           Max eq.      0.35 (0.00)    0.35 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\not\leq$ train      Obj. val.  -15.60 (0.30)  -13.83 (0.15)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\not\leq$ train/test Obj. val.  -15.60 (0.30)  -13.83 (0.15)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, no soft loss          Obj. val.  -21.78 (0.01)  -21.72 (0.03)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.   23.85 (0.52)   23.54 (0.23)   
NN                         Obj. val.  -15.66 (0.03)  -14.10 (0.05)   
                           Max eq.      0.35 (0.00)    0.35 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
NN, $\leq$ test            Obj. val.  -15.66 (0.03)  -14.10 (0.05)   
                           Max eq.      0.35 (0.00)    0.35 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
Eq.~NN                     Obj. val.  -17.07 (0.20)  -15.45 (0.09)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    1.65 (0.22)    1.79 (0.10)   
Eq.~NN, $\leq$ test        Obj. val.  -17.06 (0.20)  -15.65 (0.09)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    1.53 (0.19)    1.58 (0.08)   

                                                 50             70  \
Optimizers (OSQP, qpth)    Obj. val.  -15.05 (0.00)  -14.61 (0.00)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3                        Obj. val.  -13.46 (0.01)  -10.52 (0.93)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\neq$                Obj. val.  -12.58 (0.04)  -12.10 (0.03)   
                           Max eq.      0.35 (0.00)    0.35 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
DC3, $\not\leq$ train      Obj. val.   -1.39 (0.97)  -11.14 (0.05)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.02 (0.02)    0.00 (0.00)   
DC3, $\not\leq$ train/test Obj. val.   -1.23 (1.21)  -11.14 (0.05)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.    0.09 (0.13)    0.00 (0.00)   
DC3, no soft loss          Obj. val.  -21.84 (0.00)  -21.82 (0.01)   
                           Max eq.      0.00 (0.00)    0.00 (0.00)   
                           Max ineq.   23.83 (0.11)   23.73 (0.09)   
NN                         Obj. val.  -12.57 (0.01)  -12.12 (0.03)   
                           Max eq.      0.35 (0.00)    0.35 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
NN, $\leq$ test            Obj. val.  -12.57 (0.01)  -12.12 (0.03)   
                           Max eq.      0.35 (0.00)    0.35 (0.00)   
                           Max ineq.    0.00 (0.00)    0.00 (0.00)   
Eq.~NN                     Obj. val.   -9.16 (0.75)  -14.20 (0.06)   
                           Max eq.      0.00 (0.00)    0.0

## LaTeX 

In [ ]:
get_latex(df_simple_5050)

In [ ]:
get_latex(df_nonconvex)

In [ ]:
get_latex(df_acopf)

In [ ]:
get_latex(df_simple_varyeq)

In [ ]:
get_latex(df_simple_varyineq)